Manually tested: simpler
## Bookings data

In [0]:
# df = spark.read.format('delta') \
#         .load('/Volumes/workspace/0_flight_bronze/bronzevolume/bookings/data/')

# display(df)

In [0]:
# df.printSchema()

In [0]:
# from pyspark.sql.functions import *
# from pyspark.sql.types import *

In [0]:
# df1 =  df.withColumn('amount', col('amount').cast(DoubleType())) \
#             .withColumn('modified_date', current_timestamp()) \
#             .withColumn('booking_date', to_date(col('booking_date'))) \
#             .drop('_rescued_data')

# display(df1)

---
### Flights data:

In [0]:
df = spark.read.format('delta') \
        .load('/Volumes/workspace/0_flight_bronze/bronzevolume/flights/data/')

display(df)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:


df1 = df.withColumn('flight_date', to_date(col('flight_date')))\
        .withColumn('modified_date', current_timestamp())\
        .drop('_rescued_data')

display(df1)

---
# Customers data

In [0]:
df = spark.read.format('delta') \
        .load('/Volumes/workspace/0_flight_bronze/bronzevolume/customers/data/')

display(df.limit(100))

In [0]:
df.drop('_rescued_data')\
    .withColumn('modified_date', current_timestamp())

# Airports data

In [0]:
df = spark.read.format('delta') \
        .load('/Volumes/workspace/0_flight_bronze/bronzevolume/airports/data/')

df1 = df.drop('_rescued_data')\
        .withColumn('modified_date', current_timestamp())
        
display(df.limit(100))

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df1 = df.withColumn('modified_date', current_timestamp())\
        .drop('_rescued_data')
display(df1)

AFTER RUNNING DLT_PIPELINE TO CROSS CHECK:

In [0]:
%sql
select * from `1_flight_silver`.silver_airports limit 10;

# Main Silver flow starts here: using `dlt`

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
@dlt.table(
    name = 'stage_bookings'
)
def stage_bookings():

    df = spark.read.format('delta') \
        .load('/Volumes/workspace/0_flight_bronze/bronzevolume/bookings/data/')
    
    return df


In [0]:
@dlt.view(
    name = 'trans_bookings'
)
def trans_bookings():
    
    df = spark.readStream.table('stage_bookings')
    df1 = df.withColumn('amount', col('amount').cast(DoubleType())) \
            .withColumn('modified_date', current_timestamp()) \
            .withColumn('booking_date', to_date(col('booking_date'))) \
            .drop('_rescued_data')
    
    return df1

In [0]:
# Rules while loading the data to silver layer:

rules = {
    "rule1" : "Booking_id IS NOT NULL",
    "rule2" : "Passanger_id IS NOT NULL"
}

In [0]:
@dlt.table(
    name = 'silver_bookings'
)

@dlt.expect_all(rules)

def silver_bookings():
    df = spark.readStream.table('trans_bookings')
    return df